# Time-Frequency Consistency (TFC) + CNN PFF

In [6]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [7]:
from datetime import datetime
import lightning as L
import torch
import torchmetrics
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import CSVLogger

from minerva.analysis.metrics.balanced_accuracy import BalancedAccuracy
from minerva.analysis.model_analysis import TSNEAnalysis
from minerva.data.data_modules.har import MultiModalHARSeriesDataModule
from minerva.data.data_modules.har_rodrigues_24 import HARDataModuleCPC
from minerva.models.loaders import FromPretrained
from minerva.models.nets.lfr_har_architectures import HARSCnnEncoder
from minerva.models.ssl.tfc import TFC_Model, TFC_Backbone
from minerva.pipelines.lightning_pipeline import SimpleLightningPipeline

In [8]:
execution_id = f'run_{datetime.now().strftime("%Y%m%d-%H%M%S")}'
log_dir = f"./logs/{execution_id}" 

print(f"Execution ID: {execution_id}")
print(f"Log dir: {log_dir}")

Execution ID: run_20260127-022121
Log dir: ./logs/run_20260127-022121


## Pre-training with TFC

### 1. Defining the Data Module

In [9]:
data_module = HARDataModuleCPC(
    data_path="/workspaces/HIAAC-KR-Dev-Container/shared_data/rodrigues_2024_datasets/1-1/wisdm",
    input_size=6,
    window=60,
    overlap=60,
    batch_size=64
)

data_module

HARDataModuleCPC(batch_size=64, datasets=/workspaces/HIAAC-KR-Dev-Container/shared_data/rodrigues_2024_datasets/1-1/wisdm)

### 2. Defining the TF-C Model

In [11]:
# Get a batch of data
data_module.setup("fit")
dataset = data_module.train_dataloader()
batch_x, batch_y = next(iter(dataset))

# Create the model and forward the batch to check the output shape
# g_enc = CNN_PF_Backbone(include_middle=True)
# r = g_enc.forward(batch_x)
# r.shape

In [12]:
backbone = TFC_Backbone(
    input_channels=6,
    TS_length=60,
    single_encoding_size=128,
    time_encoder=HARSCnnEncoder(dim=2304,input_channel=6,inner_conv_output_dim=1280),
    frequency_encoder=HARSCnnEncoder(dim=2304,input_channel=6,inner_conv_output_dim=1280),
)
model = TFC_Model(
    input_channels=6,
    batch_size=64,
    # TS_length=r.shape[-1],
    TS_length=60,
    num_classes=6,
    single_encoding_size=128,
    backbone=backbone,
    pred_head=None,
)

model

shape of sb torch.Size([1, 6, 60])
shape after conv torch.Size([1, 128, 10])
shape after view torch.Size([1, 1280])
shape after mlp torch.Size([1, 2304])
shape of sb torch.Size([1, 6, 60])
shape after conv torch.Size([1, 128, 10])
shape after view torch.Size([1, 1280])
shape after mlp torch.Size([1, 2304])


[W127 02:21:44.013557976 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W127 02:21:44.015385579 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W127 02:21:44.015817928 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W127 02:21:44.026099153 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W127 02:21:44.026761144 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W127 02:21:44.027136448 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.


TFC_Model(
  (backbone): TFC_Backbone(
    (time_encoder): HARSCnnEncoder(
      (conv): Sequential(
        (0): Conv1d(6, 32, kernel_size=(8,), stride=(1,), padding=(4,), bias=False)
        (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): MaxPool1d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
        (4): Dropout(p=0.35, inplace=False)
        (5): Conv1d(32, 64, kernel_size=(8,), stride=(1,), padding=(4,), bias=False)
        (6): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (7): ReLU()
        (8): MaxPool1d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
        (9): Conv1d(64, 128, kernel_size=(8,), stride=(1,), padding=(4,), bias=False)
        (10): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (11): ReLU()
        (12): MaxPool1d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=

### 3. Defining the Pytorch Lightning Trainer Configuration

In [13]:
## Callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath='checkpoints/',
    monitor='val_loss',
    mode='min',
    save_last=True
)

## Logger
logger = CSVLogger(save_dir=log_dir, name='tfc-cnn-pff-pretraining', version=execution_id)

## Trainer
trainer = L.Trainer(
    max_epochs=3,
    accelerator="gpu",
    devices=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    # Only for testing. Remove for production. We will only train using 1 batch
    limit_train_batches=1,
    limit_val_batches=1,
)

trainer

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1)` was configured so 1 batch per epoch will be used.
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.


### 4. Creating the training pipeline (and running the training)

In [14]:
train_pipeline = SimpleLightningPipeline(
    model=model,
    trainer=trainer,
    log_dir=log_dir,
    save_run_status=True,
    seed=42
)
train_pipeline.run(data_module, task="fit")

/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/codecarbon/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]



  | Name     | Type            | Params | Mode 
-----------------------------------------------------
0 | backbone | TFC_Backbone    | 7.3 M  | train
1 | loss_fn  | NTXentLoss_poly | 0      | train
-----------------------------------------------------
7.3 M     Trainable params
0         Non-trainable params
7.3 M     Total params
29.271    Total estimated model params size (MB)
51        Modules in train mode
0         Modules in eval mode


** Seed set to: 42 **
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/codecarbon/logs/run_20260127-022121/run_2026-01-27-02-21-4563dbb1aa.yaml
Sanity Checking DataLoader 0:   0%|          | 0/1 [00:00<?, ?it/s]shape of sb torch.Size([64, 6, 60])
shape after conv torch.Size([64, 128, 10])
shape after view torch.Size([64, 1280])
shape after mlp torch.Size([64, 2304])
shape of sb torch.Size([64, 6, 60])
shape after conv torch.Size([64, 128, 10])
shape after view torch.Size([64, 1280])
shape after mlp torch.Size([64, 2304])
shape of sb torch.Size([64, 6, 60])
shape after conv torch.Size([64, 128, 10])
shape after view torch.Size([64, 1280])
shape after mlp torch.Size([64, 2304])
shape of sb torch.Size([64, 6, 60])
shape after conv torch.Size([64, 128, 10])
shape after view torch.Size([64, 1280])
shape after mlp torch.Size([64, 2304])


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:310: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 0:   0%|          | 0/1 [00:00<?, ?it/s] shape of sb torch.Size([64, 6, 60])
shape after conv torch.Size([64, 128, 10])
shape after view torch.Size([64, 1280])
shape after mlp torch.Size([64, 2304])
shape of sb torch.Size([64, 6, 60])
shape after conv torch.Size([64, 128, 10])
shape after view torch.Size([64, 1280])
shape after mlp torch.Size([64, 2304])
shape of sb torch.Size([64, 6, 60])
shape after conv torch.Size([64, 128, 10])
shape after view torch.Size([64, 1280])
shape after mlp torch.Size([64, 2304])
shape of sb torch.Size([64, 6, 60])
shape after conv torch.Size([64, 128, 10])
shape after view torch.Size([64, 1280])
shape after mlp torch.Size([64, 2304])
Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  2.18it/s, v_num=2121]shape of sb torch.Size([64, 6, 60])
shape after conv torch.Size([64, 128, 10])
shape after view torch.Size([64, 1280])
shape after mlp torch.Size([64, 2304])
shape of sb torch.Size([64, 6, 60])
shape after conv torch.Size([64, 128, 10])
shape after view t

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 1/1 [00:04<00:00,  0.22it/s, v_num=2121, val_loss=3.220]
⏱️ fit took 14.17s → saved to /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/codecarbon/logs/run_20260127-022121/timings_fit.csv
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/codecarbon/logs/run_20260127-022121/run_2026-01-27-02-21-4563dbb1aa.yaml


### 5. Inspecting checkpoints

In [15]:
ckpt_path = checkpoint_callback.last_model_path
ckpt = torch.load(ckpt_path, map_location="cuda")
ckpt = ckpt.get("state_dict", ckpt)
list(ckpt.keys())

['backbone.time_encoder.conv.0.weight',
 'backbone.time_encoder.conv.1.weight',
 'backbone.time_encoder.conv.1.bias',
 'backbone.time_encoder.conv.1.running_mean',
 'backbone.time_encoder.conv.1.running_var',
 'backbone.time_encoder.conv.1.num_batches_tracked',
 'backbone.time_encoder.conv.5.weight',
 'backbone.time_encoder.conv.6.weight',
 'backbone.time_encoder.conv.6.bias',
 'backbone.time_encoder.conv.6.running_mean',
 'backbone.time_encoder.conv.6.running_var',
 'backbone.time_encoder.conv.6.num_batches_tracked',
 'backbone.time_encoder.conv.9.weight',
 'backbone.time_encoder.conv.10.weight',
 'backbone.time_encoder.conv.10.bias',
 'backbone.time_encoder.conv.10.running_mean',
 'backbone.time_encoder.conv.10.running_var',
 'backbone.time_encoder.conv.10.num_batches_tracked',
 'backbone.time_encoder.mlp.weight',
 'backbone.time_encoder.mlp.bias',
 'backbone.frequency_encoder.conv.0.weight',
 'backbone.frequency_encoder.conv.1.weight',
 'backbone.frequency_encoder.conv.1.bias',
 'ba

## Fine-tuning with CNN PFF

### 1. Defining the Data Module

In [16]:
data_module = MultiModalHARSeriesDataModule(
    data_path="/workspaces/HIAAC-KR-Dev-Container/shared_data/daghar/standardized_view/MotionSense/",
    feature_prefixes=["accel-x", "accel-y", "accel-z", "gyro-x", "gyro-y", "gyro-z"],
    label="standard activity code",
    features_as_channels=True,
    cast_to="float32",
    batch_size=64,
)

data_module

MultiModalHARSeriesDataModule(data_path=/workspaces/HIAAC-KR-Dev-Container/shared_data/daghar/standardized_view/MotionSense, batch_size=64)

In [17]:
# Pega os dataloaders de treino e validação
data_module.setup("fit")
train_data_loader = data_module.train_dataloader()
validation_data_loader = data_module.val_dataloader()
first_batch = next(iter(train_data_loader))

X, y = first_batch
print(X.shape, y.shape)

Using DataLoader with shuffle=True
Using DataLoader with shuffle=False
torch.Size([64, 6, 60]) torch.Size([64])


### 2. Defining the CNN PFF Model for Fine-tuning

In [18]:
backbone = TFC_Backbone(
    input_channels=6,
    TS_length=60,
    single_encoding_size=128,
    time_encoder=HARSCnnEncoder(dim=2304,input_channel=6,inner_conv_output_dim=1280),
    frequency_encoder=HARSCnnEncoder(dim=2304,input_channel=6,inner_conv_output_dim=1280),
)


backbone = FromPretrained(
    model=backbone, 
    ckpt_path=checkpoint_callback.best_model_path,
    filter_keys=["backbone"],
    keys_to_rename={"backbone.": ""},      # Let's remove the prefix from the keys
                                        # on the checkpoint to load the model
                                        # correctly
    strict=True,
    error_on_missing_keys=True
)

shape of sb torch.Size([1, 6, 60])
shape after conv torch.Size([1, 128, 10])
shape after view torch.Size([1, 1280])
shape after mlp torch.Size([1, 2304])
shape of sb torch.Size([1, 6, 60])
shape after conv torch.Size([1, 128, 10])
shape after view torch.Size([1, 1280])
shape after mlp torch.Size([1, 2304])


[W127 02:22:01.283845996 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W127 02:22:01.284872995 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W127 02:22:01.285231620 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W127 02:22:01.293787840 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W127 02:22:01.294296784 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.
[W127 02:22:01.294627623 NNPACK.cpp:57] Could not initialize NNPACK! Reason: Unsupported hardware.


Performing key renaming with: {'backbone.': ''}
	Renaming key: backbone.time_encoder.conv.0.weight -> time_encoder.conv.0.weight (changed: True)
	Renaming key: backbone.time_encoder.conv.1.weight -> time_encoder.conv.1.weight (changed: True)
	Renaming key: backbone.time_encoder.conv.1.bias -> time_encoder.conv.1.bias (changed: True)
	Renaming key: backbone.time_encoder.conv.1.running_mean -> time_encoder.conv.1.running_mean (changed: True)
	Renaming key: backbone.time_encoder.conv.1.running_var -> time_encoder.conv.1.running_var (changed: True)
	Renaming key: backbone.time_encoder.conv.1.num_batches_tracked -> time_encoder.conv.1.num_batches_tracked (changed: True)
	Renaming key: backbone.time_encoder.conv.5.weight -> time_encoder.conv.5.weight (changed: True)
	Renaming key: backbone.time_encoder.conv.6.weight -> time_encoder.conv.6.weight (changed: True)
	Renaming key: backbone.time_encoder.conv.6.bias -> time_encoder.conv.6.bias (changed: True)
	Renaming key: backbone.time_encoder.co

In [19]:
from minerva.models.nets.mlp import MLP
head = MLP([256, 128, 6])
head

MLP(
  (0): Linear(in_features=256, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=6, bias=True)
)

In [20]:
from minerva.models.nets.base import SimpleSupervisedModel
model = SimpleSupervisedModel(
    backbone=backbone,
    fc=head,
    loss_fn=torch.nn.CrossEntropyLoss(),
    flatten=False,
    train_metrics = {"acc": torchmetrics.Accuracy(task="multiclass", num_classes=6)},
    val_metrics = {"acc": torchmetrics.Accuracy(task="multiclass", num_classes=6)},
    test_metrics = {"acc": torchmetrics.Accuracy(task="multiclass", num_classes=6)}
)

model

SimpleSupervisedModel(
  (backbone): TFC_Backbone(
    (time_encoder): HARSCnnEncoder(
      (conv): Sequential(
        (0): Conv1d(6, 32, kernel_size=(8,), stride=(1,), padding=(4,), bias=False)
        (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): MaxPool1d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
        (4): Dropout(p=0.35, inplace=False)
        (5): Conv1d(32, 64, kernel_size=(8,), stride=(1,), padding=(4,), bias=False)
        (6): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (7): ReLU()
        (8): MaxPool1d(kernel_size=2, stride=2, padding=1, dilation=1, ceil_mode=False)
        (9): Conv1d(64, 128, kernel_size=(8,), stride=(1,), padding=(4,), bias=False)
        (10): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (11): ReLU()
        (12): MaxPool1d(kernel_size=2, stride=2, padding=1, dilation=1

In [21]:
# Option 1: Measure ALL model parameters (including backbone AND head)
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total model parameters: {total_params:,}")
print(f"Trainable model parameters: {trainable_params:,}")

# Option 2: Measure components separately (as you intended)
# Backbone parameters
backbone_total = sum(p.numel() for p in model.backbone.parameters())
backbone_trainable = sum(p.numel() for p in model.backbone.parameters() if p.requires_grad)

# Head parameters (use model.fc, not head variable)
head_total = sum(p.numel() for p in model.fc.parameters())
head_trainable = sum(p.numel() for p in model.fc.parameters() if p.requires_grad)

print(f"\nBackbone:")
print(f"  Total parameters: {backbone_total:,}")
print(f"  Trainable parameters: {backbone_trainable:,}")

print(f"\nClassification Head:")
print(f"  Total parameters: {head_total:,}")
print(f"  Trainable parameters: {head_trainable:,}")

# Verify the sum matches Option 1
print(f"\nVerification:")
print(f"Backbone + Head total: {backbone_total + head_total:,}")
print(f"Should match total: {total_params:,}")

Total model parameters: 7,351,302
Trainable model parameters: 7,351,302

Backbone:
  Total parameters: 7,317,632
  Trainable parameters: 7,317,632

Classification Head:
  Total parameters: 33,670
  Trainable parameters: 33,670

Verification:
Backbone + Head total: 7,351,302
Should match total: 7,351,302


In [22]:
from thop import profile
evaluation_data = torch.rand(1000, 6, 60, device='cuda')
model.to('cuda')
macs, params = profile(model, inputs=(evaluation_data,))

print(f"MACs: {macs:,}")
print(f"Parameters: {params:,}")

[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[INFO] Register count_normalization() for <class 'torch.nn.modules.batchnorm.BatchNorm1d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.pooling.MaxPool1d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.dropout.Dropout'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
shape of sb torch.Size([1000, 6, 60])
shape after conv torch.Size([1000, 128, 10])
shape after view torch.Size([1000, 1280])
shape after mlp torch.Size([1000, 2304])
shape of sb torch.Size([1000, 6, 60])
shape after conv torch.Size([1000, 128, 10])
shape after view torch.Size([1000, 1280])
shape after mlp torch.Size([1000, 2304])
MACs: 10,824,704,000.0
Parameters: 7,351,302.0


In [23]:
from codecarbon import EmissionsTracker

torch.cuda.is_available()
torch.cuda.device_count()

1

In [24]:
import pandas as pd
from codecarbon import EmissionsTracker

TOTAL_RUNS = 14
DISCARD_FIRST = 2
DISCARD_LAST = 2

results = []

tracker = EmissionsTracker(
    project_name="basic_measurement",
    measure_power_secs=10,
    save_to_file=False
)

try:
    for run_id in range(TOTAL_RUNS):
        tracker.start_task(f"measure_inference_{run_id}")

        _ = model(evaluation_data)  # inference

        emissions = tracker.stop_task()

        energy_kwh = emissions.energy_consumed
        energy_mwh = energy_kwh * 1_000          # kWh → mWh
        energy_j   = energy_kwh * 3_600_000      # kWh → J
        emissions_g = emissions.emissions * 1_000 * 1_000  # kg → g→ mg

        results.append({
            "run": run_id,
            "energy_mWh": energy_mwh,
            "energy_J": energy_j,
            "emissions_mgCO2eq": emissions_g,
            # "duration_s": emissions.duration
        })

finally:
    tracker.stop()

# --- All runs ---
df_all = pd.DataFrame(results)

# --- Valid runs (ignore first & last 2) ---
df_valid = df_all.iloc[DISCARD_FIRST: TOTAL_RUNS - DISCARD_LAST]

# --- Statistics ---
mean = df_valid[["energy_mWh", "energy_J", "emissions_mgCO2eq"]].mean()
std  = df_valid[["energy_mWh", "energy_J", "emissions_mgCO2eq"]].std()

summary = pd.DataFrame(
    [mean, std],
    index=["mean", "std"]
).round(2)

# --- Final table ---
df_final = pd.concat([
    df_valid.set_index("run").round(2),
    summary
])

df_final


[codecarbon WARNING @ 02:22:03] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 02:22:03] [setup] RAM Tracking...
[codecarbon INFO @ 02:22:03] [setup] CPU Tracking...
[codecarbon WARNING @ 02:22:04] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist, and are readable, at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 02:22:04] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU E5-2630 v2 @ 2.60GHz
[codecarbon WARNING @ 02:22:04] No CPU tracking mode found. Falling back on CPU load mode.
[codecarbon INFO @ 02:22:04] [setup] GPU Tracking...
[codecarbon INFO @ 02:22:04] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 02:22:04] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: cpu_load
                GPU Tracking Method: pynvml
         

shape of sb torch.Size([1000, 6, 60])
shape after conv torch.Size([1000, 128, 10])
shape after view torch.Size([1000, 1280])
shape after mlp torch.Size([1000, 2304])
shape of sb torch.Size([1000, 6, 60])
shape after conv torch.Size([1000, 128, 10])
shape after view torch.Size([1000, 1280])
shape after mlp torch.Size([1000, 2304])


[codecarbon INFO @ 02:22:08] Delta energy consumed for CPU with cpu_load : 0.000001 kWh, power : 8.002829888 W
[codecarbon INFO @ 02:22:08] Energy consumed for All CPU : 0.000001 kWh
[codecarbon INFO @ 02:22:09] Energy consumed for all GPUs : 0.000008 kWh. Total GPU Power : 26.348768584252177 W
[codecarbon INFO @ 02:22:09] 0.000012 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 02:22:09] Energy consumed for RAM : 0.000006 kWh. RAM Power : 20.0 W


shape of sb torch.Size([1000, 6, 60])
shape after conv torch.Size([1000, 128, 10])
shape after view torch.Size([1000, 1280])
shape after mlp torch.Size([1000, 2304])
shape of sb torch.Size([1000, 6, 60])
shape after conv torch.Size([1000, 128, 10])
shape after view torch.Size([1000, 1280])
shape after mlp torch.Size([1000, 2304])


[codecarbon INFO @ 02:22:10] Delta energy consumed for CPU with cpu_load : 0.000001 kWh, power : 8.054257112 W
[codecarbon INFO @ 02:22:10] Energy consumed for All CPU : 0.000003 kWh
[codecarbon INFO @ 02:22:10] Energy consumed for all GPUs : 0.000016 kWh. Total GPU Power : 26.612262695775566 W
[codecarbon INFO @ 02:22:10] 0.000025 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 02:22:10] Energy consumed for RAM : 0.000010 kWh. RAM Power : 20.0 W


...

shape of sb torch.Size([1000, 6, 60])
shape after conv torch.Size([1000, 128, 10])
shape after view torch.Size([1000, 1280])
shape after mlp torch.Size([1000, 2304])
shape of sb torch.Size([1000, 6, 60])
shape after conv torch.Size([1000, 128, 10])
shape after view torch.Size([1000, 1280])
shape after mlp torch.Size([1000, 2304])


[codecarbon INFO @ 02:22:21] Delta energy consumed for CPU with cpu_load : 0.000001 kWh, power : 8.012644352 W
[codecarbon INFO @ 02:22:21] Energy consumed for All CPU : 0.000016 kWh
[codecarbon INFO @ 02:22:21] Energy consumed for all GPUs : 0.000100 kWh. Total GPU Power : 26.51302585492331 W
[codecarbon INFO @ 02:22:21] 0.000157 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 02:22:22] Energy consumed for RAM : 0.000044 kWh. RAM Power : 20.0 W


shape of sb torch.Size([1000, 6, 60])
shape after conv torch.Size([1000, 128, 10])
shape after view torch.Size([1000, 1280])
shape after mlp torch.Size([1000, 2304])
shape of sb torch.Size([1000, 6, 60])
shape after conv torch.Size([1000, 128, 10])
shape after view torch.Size([1000, 1280])
shape after mlp torch.Size([1000, 2304])


[codecarbon INFO @ 02:22:22] Delta energy consumed for CPU with cpu_load : 0.000001 kWh, power : 8.093242088 W
[codecarbon INFO @ 02:22:22] Energy consumed for All CPU : 0.000018 kWh
[codecarbon INFO @ 02:22:22] Energy consumed for all GPUs : 0.000107 kWh. Total GPU Power : 22.440017820878843 W
[codecarbon INFO @ 02:22:22] 0.000168 kWh of electricity and 0.000000 L of water were used since the beginning.
[codecarbon INFO @ 02:22:22] Energy consumed for RAM : 0.000044 kWh. RAM Power : 20.0 W
[codecarbon INFO @ 02:22:23] Delta energy consumed for CPU with cpu_load : 0.000000 kWh, power : 8.007475256000001 W
[codecarbon INFO @ 02:22:23] Energy consumed for All CPU : 0.000018 kWh
[codecarbon INFO @ 02:22:23] Energy consumed for all GPUs : 0.000115 kWh. Total GPU Power : 56.96237977906044 W
[codecarbon INFO @ 02:22:23] 0.000176 kWh of electricity and 0.000000 L of water were used since the beginning.


,energy_mWh,energy_J,emissions_mgCO2eq
2,0.01,39.67,1.08
3,0.01,44.17,1.21
4,0.01,44.42,1.21
5,0.01,44.51,1.22
6,0.01,39.76,1.09
7,0.01,43.96,1.20
8,0.01,44.81,1.22
9,0.01,39.67,1.08
10,0.01,44.73,1.22
11,0.01,44.13,1.21


In [25]:
a

NameError: name 'a' is not defined

### 3. Defining the Pytorch Lightning Trainer Configuration

In [ ]:
## Callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints/", monitor="val_loss", mode="min", save_last=True
)

## Logger
logger = CSVLogger(
    save_dir=log_dir, name="tfc-cnn-pff-finetune", version=execution_id
)

## Trainer
trainer = L.Trainer(
    max_epochs=3,
    accelerator="gpu",
    devices=1,
    logger=logger,
    callbacks=[checkpoint_callback],
    # Only for testing. Remove for production. We will only train using 1 batch
    limit_train_batches=1,
    limit_val_batches=1,
)

trainer

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
`Trainer(limit_train_batches=1)` was configured so 1 batch per epoch will be used.
`Trainer(limit_val_batches=1)` was configured so 1 batch will be used.


### 4. Creating the fine-tuning pipeline (and running the training)

In [ ]:
train_pipeline = SimpleLightningPipeline(
    model=model,
    trainer=trainer,
    log_dir=log_dir,
    save_run_status=True,
    seed=42
)
train_pipeline.run(data_module, task="fit")

Seed set to 42





 cuda device: 


 NVIDIA RTX A5000
Log directory set to: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/logs/run_20250218-190101
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/logs/run_20250218-190101/run_2025-02-18-19-02-1401fdfefffd4744888e3d3786c55473e2.yaml


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/callbacks/model_checkpoint.py:652: Checkpoint directory /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]

  | Name     | Type             | Params | Mode 
------------------------------------------------------
0 | backbone | TFC_Backbone     | 553 K  | train
1 | fc       | MLP              | 33.7 K | train
2 | loss_fn  | CrossEntropyLoss | 0      | train
------------------------------------------------------
587 K     Trainable params
0         Non-trainable params
587 K     Total params
2.350     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]Using DataLoader with shuffle=False
Using DataLoader with shuffle=True                                         


/usr/local/lib/python3.10/dist-packages/lightning/pytorch/loops/fit_loop.py:298: The number of training batches (1) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 2: 100%|██████████| 1/1 [00:08<00:00,  0.12it/s, v_num=0101, val_loss=2.730, val_acc=0.000, train_loss=1.410, train_acc=0.422]

`Trainer.fit` stopped: `max_epochs=3` reached.


Epoch 2: 100%|██████████| 1/1 [00:08<00:00,  0.12it/s, v_num=0101, val_loss=2.730, val_acc=0.000, train_loss=1.410, train_acc=0.422]
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/logs/run_20250218-190101/run_2025-02-18-19-02-1401fdfefffd4744888e3d3786c55473e2.yaml


## Evaluating the Fine-tuned Model

In [ ]:
test_pipeline = SimpleLightningPipeline(
    model=model,
    trainer=trainer,
    log_dir=log_dir,
    save_run_status=True,
    seed=42,
    classification_metrics={
        "accuracy": torchmetrics.Accuracy(num_classes=6, task="multiclass"),
        "f1": torchmetrics.F1Score(num_classes=6, task="multiclass"),
        "precision": torchmetrics.Precision(num_classes=6, task="multiclass"),
        "recall": torchmetrics.Recall(num_classes=6, task="multiclass"),
        "balanced_accuracy": BalancedAccuracy(num_classes=6, task="multiclass"),
    },
    apply_metrics_per_sample=False,
    # model_analysis={
    #     "tsne": TSNEAnalysis(
    #         height=800,
    #         width=800,
    #         legend_title="Activity",
    #         title="t-SNE of TFC Finetuned on MotionSense",
    #         output_filename="tsne_tfc_finetuned_motionsense.pdf",
    #         label_names={
    #             0: "sit",
    #             1: "stand",
    #             2: "walk",
    #             3: "stair up",
    #             4: "stair down",
    #             5: "run",
    #             6: "stair up and down",
    #         },
    #     )
    # },
)

test_pipeline.run(
    data_module, task="evaluate", ckpt_path=checkpoint_callback.best_model_path
)

Seed set to 42





 cuda device: 


 NVIDIA RTX A5000
Log directory set to: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/logs/run_20250218-190101
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/logs/run_20250218-190101/run_2025-02-18-19-02-42b7807a4e71cb4e9fb004c1ebe8346112.yaml
Using DataLoader with shuffle=False


Restoring states from the checkpoint path at /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/checkpoints/epoch=0-step=1-v3.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [3]
Loaded model weights from the checkpoint at /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/checkpoints/epoch=0-step=1-v3.ckpt


Using DataLoader with shuffle=False
Predicting DataLoader 0: 100%|██████████| 17/17 [00:00<00:00, 103.75it/s]
Running classification metrics...
Metrics saved to /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/logs/run_20250218-190101/metrics_2025-02-18-19-02-42b7807a4e71cb4e9fb004c1ebe8346112.yaml
Pipeline info saved at: /workspaces/HIAAC-KR-Dev-Container/Minerva-Exps/benchmarks/experiments/docs/tfc/logs/run_20250218-190101/run_2025-02-18-19-02-42b7807a4e71cb4e9fb004c1ebe8346112.yaml


/workspaces/HIAAC-KR-Dev-Container/Minerva-Dev/minerva/analysis/metrics/balanced_accuracy.py:59: UserWarning: y_pred contains nan values and not all classes passed
  warnings.warn(f"y_pred contains nan values and not all classes passed")


{'classification': {'accuracy': [0.32862523198127747],
  'f1': [0.32862523198127747],
  'precision': [0.32862523198127747],
  'recall': [0.32862523198127747],
  'balanced_accuracy': [0.24832433462142944]}}